# test note


* jupyterはコンテナ起動すること
* テストベッド一式起動済みであること


In [1]:
!pip install --upgrade pip
!pip install --upgrade --no-deps --force-reinstall ../lib/ait_sdk-0.1.3-py3-none-any.whl

Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.2.3)
Processing /workdir/root/lib/ait_sdk-0.1.3-py3-none-any.whl
  Attempting uninstall: ait-sdk
    Found existing installation: ait-sdk 0.1.3
    Uninstalling ait-sdk-0.1.3:
      Successfully uninstalled ait-sdk-0.1.3


In [2]:
from pathlib import Path
import pprint
from ait_sdk.test.hepler import Helper

In [3]:
# settings cell

# mounted dir
root_dir = Path('/workdir/root/ait')

ait_name='eval_mnist_acc_tf2.3'
ait_version='0.1'

ait_full_name=f'{ait_name}_{ait_version}'
ait_dir = root_dir / ait_full_name

td_name=f'{ait_name}_test'

# (dockerホスト側の)インベントリ登録用アセット格納ルートフォルダ
invenotory_root_dir=r'C:\m-akita\github\qai-testbed\exp\788_AIT_Dev_refactor\ait\eval_mnist_acc_tf2.3_0.1\local_qai\inventory'

# entry point address
# コンテナ起動かどうかでポート番号が変わるため、切り替える
is_container = True
if is_container:
    backend_entry_point = 'http://host.docker.internal:8888/qai-testbed/api/0.0.1'
    ip_entry_point = 'http://host.docker.internal:8888/qai-ip/api/0.0.1'
else:
    backend_entry_point = 'http://host.docker.internal:5000/qai-testbed/api/0.0.1'
    ip_entry_point = 'http://host.docker.internal:6000/qai-ip/api/0.0.1'

# aitのデプロイフラグ
# 一度実施すれば、それ以降は実施しなくてＯＫ
is_init_ait = True

# インベントリの登録フラグ
# 一度実施すれば、それ以降は実施しなくてＯＫ
is_init_inventory = True


In [4]:
helper = Helper(backend_entry_point=backend_entry_point, 
                ip_entry_point=ip_entry_point,
                ait_dir=ait_dir,
                ait_full_name=ait_full_name)

In [5]:
# health check

helper.get_bk('/health-check')
helper.get_ip('/health-check')

<Response [200]>
{'Code': 0, 'Message': 'alive.'}
<Response [200]>
{'Code': 0, 'Message': 'alive.'}


<Response [200]>

In [6]:
# deploy AIT
if is_init_ait:
    helper.deploy_ait_non_build()
else:
    print('skip deploy AIT')

<Response [400]>
{'Code': 'T54000', 'Message': 'not found test descriptions'}
<Response [200]>
{'Code': 'D00001', 'Message': 'Deploy success'}


In [7]:
# add inventories

if is_init_inventory:
    inv1_name = helper.post_inventory('trained_model_1', 'model', 'UNIX_FILE_SYSTEM', 
                                      f'{invenotory_root_dir}\\trained_model\\model_1.h5',
                                      'Tensorflow 2.3で学習したモデル_1', ['h5'])
    inv2_name = helper.post_inventory('test_set_images', 'dataset', 'UNIX_FILE_SYSTEM', 
                                      f'{invenotory_root_dir}\\test_set_images\\t10k-images-idx3-ubyte.gz',
                                      'テスト画像セット（MNISTフォーマット）', ['gz'])
    inv3_name = helper.post_inventory('test_set_labels', 'dataset', 'UNIX_FILE_SYSTEM', 
                                      f'{invenotory_root_dir}\\test_set_labels\\t10k-labels-idx1-ubyte.gz',
                                      'テスト画像ラベル（MNISTフォーマット）', ['gz'])
else:
    print('skip add inventories')

<Response [200]>
{'result': {'Code': 'I22000', 'Message': 'append Inventory success.'}}
<Response [200]>
{'result': {'Code': 'I22000', 'Message': 'append Inventory success.'}}
<Response [200]>
{'result': {'Code': 'I22000', 'Message': 'append Inventory success.'}}


In [8]:
# get ait_json and inventory_jsons

res_json = helper.get_bk('/QualityMeasurements/RelationalOperators', is_print_json=False).json()
eq_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '=='][0])
nq_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '!='][0])
gt_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '>'][0])
ge_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '>='][0])
lt_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '<'][0])
le_id = int([r['Id'] for r in res_json['RelationalOperator'] if r['Expression'] == '<='][0])

res_json = helper.get_bk('/testRunners', is_print_json=False).json()
ait_json = [j for j in res_json['TestRunners'] if j['Name'] == ait_name][-1]

inv_1_json = helper.get_inventory(inv1_name)
inv_2_json = helper.get_inventory(inv2_name)
inv_3_json = helper.get_inventory(inv3_name)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [9]:
# add teast_descriptions

helper.post_td(td_name, 5,
               quality_measurements=[
                {"Id":ait_json['Report']['Measures'][0]['Id'], "Value":"0.95", "RelationalOperatorId":gt_id},
                {"Id":ait_json['Report']['Measures'][1]['Id'], "Value":"0.95", "RelationalOperatorId":gt_id},
                {"Id":ait_json['Report']['Measures'][2]['Id'], "Value":"0.95", "RelationalOperatorId":gt_id},
                {"Id":ait_json['Report']['Measures'][3]['Id'], "Value":"0.95", "RelationalOperatorId":gt_id},
                {"Id":ait_json['Report']['Measures'][4]['Id'], "Value":"0.95", "RelationalOperatorId":gt_id},
                {"Id":ait_json['Report']['Measures'][5]['Id'], "Value":"0.95", "RelationalOperatorId":gt_id},
                {"Id":ait_json['Report']['Measures'][6]['Id'], "Value":"0.95", "RelationalOperatorId":gt_id},
                {"Id":ait_json['Report']['Measures'][7]['Id'], "Value":"0.95", "RelationalOperatorId":gt_id},
                {"Id":ait_json['Report']['Measures'][8]['Id'], "Value":"0.95", "RelationalOperatorId":gt_id}
               ],
               target_inventories=[
                {"Id":1, "InventoryId": inv_1_json['Id'], "TemplateInventoryId": ait_json['TargetInventories'][0]['Id']},
                {"Id":2, "InventoryId": inv_2_json['Id'], "TemplateInventoryId": ait_json['TargetInventories'][1]['Id']},
                {"Id":2, "InventoryId": inv_3_json['Id'], "TemplateInventoryId": ait_json['TargetInventories'][2]['Id']}
               ],
               test_runner={
                "Id":ait_json['Id'],
                "Params":[
                    {"TestRunnerParamTemplateId":ait_json['ParamTemplates'][0]['Id'], "Value":"10"},
                    {"TestRunnerParamTemplateId":ait_json['ParamTemplates'][1]['Id'], "Value":"28"},
                    {"TestRunnerParamTemplateId":ait_json['ParamTemplates'][2]['Id'], "Value":"macro"},
                    {"TestRunnerParamTemplateId":ait_json['ParamTemplates'][3]['Id'], "Value":"raise"}
                ]
               })

<Response [200]>
{'Result': {'Code': 'T22000', 'Message': 'append test description success.'}}


In [10]:
# get test_description_jsons
td_1_json = helper.get_td(td_name)

<Response [200]>


In [11]:
# run test_descriptions
helper.post_run_and_wait(td_1_json['Id'])

<Response [200]>
{'Job': {'Id': '2', 'StartDateTime': '2020-09-15 16:48:30.340578+09:00'},
 'Result': {'Code': 'R12000', 'Message': 'job launch success.'}}
[{'Id': 2,
  'Result': 'OK',
  'ResultDetail': '{\n'
                  '  "Accuracy": true,\n'
                  '  "Precision": true,\n'
                  '  "Recall": true,\n'
                  '  "F−measure": true,\n'
                  '  "AccuracyByClass": true,\n'
                  '  "PrecisionByClass": true,\n'
                  '  "RecallByClass": true,\n'
                  '  "F−measureByClass": true,\n'
                  '  "AUC": true\n'
                  '}',
  'Status': 'DONE',
  'TestDescriptionID': 3}]


In [12]:
res_json = helper.get_td_detail(td_1_json['Id'])
pprint.pprint(res_json)

<Response [200]>
{'Result': {'Code': 'T32000', 'Message': 'get detail success.'},
 'TestDescriptionDetail': {'Id': 3,
                           'Name': 'eval_handwrite_acc_tf2.3_test',
                           'Opinion': '',
                           'QualityDimension': {'Id': 5, 'Name': '機械学習モデルの正確性'},
                           'QualityMeasurements': [{'Description': '(TP+TN) / '
                                                                   '(TP+FP+FN+TN)',
                                                    'Id': 2,
                                                    'Name': 'Accuracy',
                                                    'RelationalOperatorId': 3,
                                                    'Structure': 'single',
                                                    'Value': '0.95'},
                                                   {'Description': 'TP / '
                                                                   '(TP+FP)',
                 

In [13]:
# generate report
res = helper.post_report(td_1_json['Id'])
pprint.pprint(res)

<Response [200]>
{'OutParams': {'ReportUrl': 'http://127.0.0.1:8888/qai-testbed/api/0.0.1/download/20'},
 'Result': {'Code': 'D12000', 'Message': 'command invoke success.'}}
<bound method Response.json of <Response [200]>>
